In [ ]:
!pip install numpy

In [ ]:
!pip install pandas gensim

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
# import gensim

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
# from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
eng_stopw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
punctuation = string.punctuation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset_AOL_NLP/train (2).csv', delimiter=';')

In [ ]:
print(dataset.isnull().sum())

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64


In [ ]:
dataset['label'].fillna(dataset['label'].mode()[0] , inplace = True)

<ipython-input-8-8d661ca88275>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['label'].fillna(dataset['label'].mode()[0] , inplace = True)


Terdapat 1 kolom label yang kosong, maka akan kita isi menggunakan mode

In [ ]:
print(dataset.isnull().sum())

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64


In [ ]:
frekuensi_label = dataset["label"].value_counts()
print(frekuensi_label)

label
1    13246
0    11107
Name: count, dtype: int64


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24353 entries, 0 to 24352
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  24353 non-null  int64 
 1   title       24353 non-null  object
 2   text        24353 non-null  object
 3   label       24353 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 761.2+ KB


In [ ]:
dataset.drop(columns=['Unnamed: 0'] , inplace=True)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24353 entries, 0 to 24352
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   24353 non-null  object
 1   text    24353 non-null  object
 2   label   24353 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 570.9+ KB


In [ ]:
duplicates = dataset.duplicated()

In [ ]:
print(f"Jumlah duplikat: {duplicates.sum()}")
dataset_cleaned = dataset.drop_duplicates()
dataset_cleaned = dataset_cleaned.reset_index(drop=True)
print(dataset_cleaned.index)

Jumlah duplikat: 1
RangeIndex(start=0, stop=24352, step=1)


In [ ]:
x = dataset['title']
y = dataset['text']
print(x.head())
print('======================================================')
print(y.head())

0    Palestinians switch off Christmas lights in Be...
1    China says Trump call with Taiwan president wo...
2     FAIL! The Trump Organization’s Credit Score W...
3    Zimbabwe military chief's China trip was norma...
4    THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...
Name: title, dtype: object
0    RAMALLAH, West Bank (Reuters) - Palestinians s...
1    BEIJING (Reuters) - U.S. President-elect Donal...
2    While the controversy over Trump s personal ta...
3    BEIJING (Reuters) - A trip to Beijing last wee...
4    There has never been a more UNCOURAGEOUS perso...
Name: text, dtype: object


In [ ]:
def preprocessing_sentence(text):
  words = word_tokenize(text.lower())

  words = [w for w in words if w not in eng_stopw and w not in punctuation and w.isalpha()]
  words = [lemmatizer.lemmatize(word) for word in words]

  return words

In [ ]:
clean_text = dataset_cleaned['text'].apply(preprocessing_sentence)
clean_title = dataset_cleaned['title'].apply(preprocessing_sentence)

In [ ]:
print(clean_text)
print(clean_title)

0        [ramallah, west, bank, reuters, palestinian, s...
1        [beijing, reuters, donald, trump, call, taiwan...
2        [controversy, trump, personal, tax, return, co...
3        [beijing, reuters, trip, beijing, last, week, ...
4        [never, uncourageous, person, white, house, ba...
                               ...                        
24347    [mexico, city, reuters, key, committee, mexico...
24348    [toast, bigger, trouble, thought, much, whole,...
24349    [kremlin, nato, created, agression, russia, to...
24350    [dallas, cowboy, star, wide, receiver, dez, br...
24351    [update, nordstrom, stock, closed, slightly, t...
Name: text, Length: 24352, dtype: object
0        [palestinian, switch, christmas, light, bethle...
1        [china, say, trump, call, taiwan, president, w...
2        [fail, trump, organization, credit, score, mak...
3        [zimbabwe, military, chief, china, trip, norma...
4        [uncourageous, president, ever, receives, cour...
               

In [ ]:
X = pd.DataFrame({'title': clean_title, 'text': clean_text})
y = dataset_cleaned['label']

In [ ]:
X['combined_text'] = X['title'] + X['text']

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [ ]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
X_train['combined_text'] = X_train['title'] + X_train['text']
X_val['combined_text'] = X_val['title'] + X_val['text']
X_test['combined_text'] = X_test['title'] + X_test['text']

# Model + Vectorizer -> BERT

In [ ]:
import transformers
import torch

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Pastikan semua data type adalah string

In [ ]:
X_train['combined_text'] = X_train['combined_text'].astype(str)
X_val['combined_text'] = X_val['combined_text'].astype(str)
X_test['combined_text'] = X_test['combined_text'].astype(str)

In [ ]:
train_encodings = tokenizer(list(X_train['combined_text']), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(list(X_val['combined_text']), truncation=True, padding=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(list(X_test['combined_text']), truncation=True, padding=True, max_length=512, return_tensors='pt')

In [ ]:
class HoaxDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)

train_dataset = HoaxDataset(train_encodings, y_train)
val_dataset = HoaxDataset(val_encodings, y_val)
test_dataset = HoaxDataset(test_encodings, y_test)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for name, param in model.bert.named_parameters():
    if not any(layer in name for layer in ['encoder.layer.10', 'encoder.layer.11']):
        param.requires_grad = False

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    run_name='bert-hoax-classification-run-2'
)
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds)
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ian-frderick (ian-frderick-binus-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.066900,0.078950,0.980564,0.981966
1000,0.068800,0.048205,0.984396,0.985522
1500,0.092200,0.059995,0.980016,0.981607
2000,0.091900,0.058827,0.981385,0.982949
2500,0.003300,0.055974,0.984123,0.985279
3000,0.080600,0.055071,0.986039,0.987111
3500,0.106200,0.059430,0.986039,0.987111
4000,0.033900,0.062352,0.981933,0.983434
4500,0.000600,0.072742,0.982480,0.983944
5000,0.004600,0.067407,0.983849,0.985165


TrainOutput(global_step=5330, training_loss=0.05772812833448638, metrics={'train_runtime': 4532.5588, 'train_samples_per_second': 18.804, 'train_steps_per_second': 1.176, 'total_flos': 2.24249552483328e+16, 'train_loss': 0.05772812833448638, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.04342227056622505,
 'eval_accuracy': 0.9854913769504516,
 'eval_f1': 0.9866330390920555,
 'eval_runtime': 104.0766,
 'eval_samples_per_second': 35.099,
 'eval_steps_per_second': 1.105,
 'epoch': 5.0}

In [ ]:
model.save_pretrained('/content/drive/MyDrive/model_hoax_trained_final')
tokenizer.save_pretrained('/content/drive/MyDrive/model_hoax_trained_final')

('/content/drive/MyDrive/model_hoax_trained_final/tokenizer_config.json',
 '/content/drive/MyDrive/model_hoax_trained_final/special_tokens_map.json',
 '/content/drive/MyDrive/model_hoax_trained_final/vocab.txt',
 '/content/drive/MyDrive/model_hoax_trained_final/added_tokens.json')

In [ ]:
save_directory = "/content/drive/MyDrive/Colab Notebooks/Dataset_AOL_NLP/model_hoax_BERT_final"
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/Colab Notebooks/Dataset_AOL_NLP/model_hoax_BERT_final/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Dataset_AOL_NLP/model_hoax_BERT_final/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Dataset_AOL_NLP/model_hoax_BERT_final/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Dataset_AOL_NLP/model_hoax_BERT_final/added_tokens.json')

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast

model = BertForSequenceClassification.from_pretrained(save_directory)
tokenizer = BertTokenizerFast.from_pretrained(save_directory)

In [ ]:
trainer = Trainer(model=model)
predictions = trainer.predict(test_dataset)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


In [ ]:
y_pred_logits = predictions.predictions
y_true = predictions.label_ids

y_pred = np.argmax(y_pred_logits, axis=1)


acc = accuracy_score(y_true, y_pred)
print(f"Akurasi: {acc:.4f}")

Akurasi: 0.9830


In [ ]:
import os
os.listdir(save_directory)

['config.json',
 'model.safetensors',
 'training_args.bin',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'vocab.txt']